### Анализ данных сервиса для онлайн чтения книг. 

#### Краткое описание. 

Компания в которой мы работаем, в отделе аналитики, приобрела крупный сервис для чтения книг по подписке. Мы получили доступ к БД нового сервиса, которая содержит информацию о книгах, издательствах, авторах, а также пользовательские обзоры книг. Нам необходимо провести первичный анализ БД и сформулировать ценностное предложение для нового продукта.

#### Описание полученных данных.

- Таблица **"books"** - *Содержит данные о книгах*:
   - book_id — идентификатор книги;
   - author_id — идентификатор автора;
   - title — название книги;
   - num_pages — количество страниц;
   - publication_date — дата публикации книги;
   - publisher_id — идентификатор издателя.

- Таблица **"authors"** - *Содержит данные об авторах*:
   - author_id — идентификатор автора;
   - author — имя автора.

- Таблица **"publishers"** - *Содержит данные об издательствах*:
   - publisher_id — идентификатор издательства;
   - publisher — название издательства;

- Таблица **"ratings"** - *Содержит данные о пользовательских оценках книг*:
   - rating_id — идентификатор оценки;
   - book_id — идентификатор книги;
   - username — имя пользователя, оставившего оценку;
   - rating — оценка книги.

- Таблица **"reviews"** - *Содержит данные о пользовательских обзорах*:
   - review_id — идентификатор обзора;
   - book_id — идентификатор книги;
   - username — имя автора обзора;
   - text — текст обзора.

#### Подключение к БД.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
 'pwd': 'Sdf4$2;d-d30pp', # пароль
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
 'port': 6432, # порт подключения
 'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])

In [3]:
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [4]:
# создаем функцию для вывода запросов
def output(query):
  return pd.io.sql.read_sql(query, con = engine)

После подключения исследуем таблицы, для начала просмотрим первые строки в каждой таблице. 

#### Обзор данных.

In [5]:
# просматриваем первые 5 строк таблицы "books"
query = '''
SELECT *
FROM books
LIMIT 5;
        '''
# выводим запрос
output(query)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [6]:
# просматриваем первые 5 строк таблицы "authors"
query = '''
SELECT *
FROM authors
LIMIT 5;
        '''

# выводим запрос
output(query)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [7]:
# просматриваем первые 5 строк таблицы "ratings"
query = '''
SELECT *
FROM ratings
LIMIT 5;
        '''

# выводим запрос
output(query)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [8]:
# просматриваем первые 5 строк таблицы "reviews"
query = '''
SELECT *
FROM reviews
LIMIT 5;
        '''

# выводим запрос
output(query)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [9]:
# просматриваем первые 5 строк таблицы "publishers"
query = '''
SELECT *
FROM publishers
LIMIT 5;
        '''

# выводим запрос
output(query)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Получили общее представление о данных.

#### Анализ данных. 

##### Посчитаем, сколько книг вышло после 1 января 2000 года.

In [10]:
# формируем запрос к таблице "books"
query = '''
SELECT COUNT(book_id) AS count_of_books
FROM books
WHERE publication_date > '2000-01-01';
        '''

# выводим запрос
output(query)

,count_of_books
0,819


Общее кол-во книг вышедших после 1 января 2000 года и представленных в сервисе - 819. 

##### Посчитаем количество обзоров и среднюю оценку для каждой книги. 

В запросе оставим уникальные id книг чтобы избежать дублей в названии книг.

In [11]:
# формируем запрос к таблицам: books, reviews, ratings
query = '''
SELECT b.book_id,
       b.title,
       COUNT(DISTINCT(r.review_id)) AS count_review,
       AVG(rt.rating) AS average_rating
FROM books AS b
LEFT JOIN reviews AS r ON r.book_id = b.book_id
LEFT JOIN ratings AS rt ON rt.book_id = b.book_id
GROUP BY 1
;
        '''

# выводим запрос
output(query)

,book_id,title,count_review,average_rating
0,1,'Salem's Lot,2,3.666667
1,2,1 000 Places to See Before You Die,1,2.500000
2,3,13 Little Blue Envelopes (Little Blue Envelope...,3,4.666667
3,4,1491: New Revelations of the Americas Before C...,2,4.500000
4,5,1776,4,4.000000
...,...,...,...,...
995,996,Wyrd Sisters (Discworld #6; Witches #2),3,3.666667
996,997,Xenocide (Ender's Saga #3),3,3.400000
997,998,Year of Wonders,4,3.200000
998,999,You Suck (A Love Story #2),2,4.500000


Как мы можем заметить на первом месте книга - "Twilight (Twilight #1)", исходя из названия можем предположить, что существует книга #2, а так же возможно от иных издателей, что может искажать информацию. Можем сказать, что наибольшее кол-во книг получает около 6 оценок. 	

##### Определим издательство, которое выпустило наибольшее число книг толще 50 страниц.

Данный запрос поможет исключить нам брошюры из анализа. 

In [12]:
# формируем запрос к таблицам: books, publishers
query = '''
SELECT p.publisher,
       COUNT(b.book_id)
FROM publishers as p
LEFT JOIN books as b ON p.publisher_id = b.publisher_id
WHERE b.num_pages > 50
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10;
        '''

# выводим запрос
output(query)

,publisher,count
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Ballantine Books,19
5,Bantam,19
6,Berkley,17
7,St. Martin's Press,14
8,Berkley Books,14
9,William Morrow Paperbacks,13


В списке книг которые выпускают наибольшее кол-во книг толще 50 страниц несомненый лидер издательство - "Penguin Books" выпустившее более 40 книг. 

##### Определим автора с самой высокой средней оценкой книг.

В данном запросе будем учитывать только книги с 50 и более оценками.

In [13]:
query = '''
SELECT author,
       AVG(average_ratings) AS average_rating 
FROM (
  SELECT a.author,
         b.book_id,
         AVG(r.rating) AS average_ratings
  FROM authors AS a
  LEFT JOIN books AS b ON b.author_id = a.author_id
  LEFT JOIN ratings AS r ON r.book_id = b.book_id
  GROUP BY 1, 2
  HAVING COUNT(rating) > 50) AS time_table
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10;
        '''

# выводим запрос
output(query)

,author,average_rating
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Dan Brown,3.754540


Как мы можем заметить средняя оценка примерно одинаковая между книгами с большим кол-вом оценок. (более 50). Однако, что интересно, что часть книг написана в соавторстве и делать выводы какой-то писатель лучше продается преждевременно.

##### Посчитаем среднее количество обзоров.

В данном запросе будем учитывать только пользователей, которые поставили больше 50 оценок.

In [14]:
query = '''
SELECT ROUND(AVG(count_reviews)) AS average_count_reviews
FROM (
  SELECT username,
         COUNT(review_id) as count_reviews
  FROM reviews
  WHERE username IN (
      SELECT username
      FROM ratings
      GROUP BY 1
      HAVING COUNT(ratings) > 50)
GROUP BY 1
ORDER BY 2 DESC) AS cnt_users
        '''

# выводим запрос
output(query)

,average_count_reviews
0,24.0


Активный пользователь, поставивший более 50 оценок на сайте в среднем для каждой 2ой прочитанной книги оставляет обзор, что весьма существенно. 

#### Общий вывод.

- Проведен краткий анализ полученных данных нового сервиса в нашей компании: 
  - В сервисе представлено 819 книг вышедших после 1 января 2000 года;
  - Наибольшее кол-во обзоров получила книга "Twilight (Twilight #1)", однако мы отметили, что данная книга скорее всего не единственная с данным названием, что может искажать рез-ты среднего рейтинга, ведь на 1 книгу с одинаковым названием могут быть разные рейтинги;
  - Нибольшее кол-во книг толще 50 страниц выпустило издательство "Penguin Books" - 42 книги, далее следует "Vintage" с 31 книгой. 
  - Автор с наиболее высокой средней оценкой - J.K. Rowling/Mary GrandPré - 4.3. 
  - Наиболее активные пользователи сервиса оставляют рецензию на каждое 2ую прочитанную книгу. 
  
--- 

- Рекомендации:
  - Необходимо пересмотреть названия книг, т.к мы поняли могут быть дубли, в связи с разным изданием, нумерацией и т.д., что может исказить среднюю оценку для книги. Думаю решением в данном случае будет ввести категоризацию по жанрам для книг для сравнений рез-тов; 
  - Наибольшее кол-во книг выпустило издательство "Penguin Books" - 42 книги, думаю стоит просмотреть рейтинги внутри данного издательства и например внутри издательства "Vintage", вдруг какие-то книги с одинаковым названием, но разным издательством продаются лучше и получают оценки лучше;
  - Автор получивший наивысшую среднюю оценку - J.K. Rowling/Mary GrandPré - 4.3. Однако стоит понимать, что многие книги пишутся в соавторстве, а порой получают просто переиздание, что так же может искажать среднюю оценку этот момент будет более понятен, когда в одном из будущих исследований мы сделаем жанры и проведем исследование рейтингов между издательствами; 
  - Интересно, что активные пользователи оставляют большое кол-во рецензий, думаю можно составить рейтинг рецензий, где пользователи будут оценивать друг друга и вести дискуссии (с модерацией), что-то вроде клуба книголюбов внутри нашего сервиса. 